In [1]:
from pathlib import Path
import json
import pandas as pd
from PIL import Image

In [2]:
wd = Path.cwd().parent
data_dir = (wd / 'data/Rad_images').resolve()

In [3]:
data_dir

PosixPath('/home/xiaofey/xray/xray-master/data/Rad_images')

In [4]:
csv_nifti_path = (wd / 'data/nifti.csv').resolve().as_posix()
jpg_export_path = (wd / 'data/exported_jpgs').resolve().as_posix()
lbx_path = (wd / 'data/lbx_data.json').resolve().as_posix()

In [5]:
df = pd.read_csv(csv_nifti_path)

In [6]:
df

,PATIENT_ID,FULL_PATH,FILENAME_VIEW,MIN,MAX,DATA_TYPE,WIDTH,HEIGHT,NUM_CHANNELS,ROI,JPG_PATH
0,AN00121,/zfs/wficai/xray/xray//Images/AN (121)/Shoulde...,Shoulder_L_1,0,32767,int16,2566,1779,1,False,/home/xiaofey/xray/xray-master/data/exported_j...
1,AN00121,/zfs/wficai/xray/xray//Images/AN (121)/Shoulde...,Shoulder_L_3_ROI1,0,1,uint8,1057,1605,1,True,/home/xiaofey/xray/xray-master/data/exported_j...
2,AN00121,/zfs/wficai/xray/xray//Images/AN (121)/Shoulde...,Shoulder_L_2,0,32767,int16,2153,1844,1,False,/home/xiaofey/xray/xray-master/data/exported_j...
3,AN00121,/zfs/wficai/xray/xray//Images/AN (121)/Shoulde...,Shoulder_L_3,0,29119,int16,1057,1605,1,False,/home/xiaofey/xray/xray-master/data/exported_j...
4,AN00308,/zfs/wficai/xray/xray//Images/AN (308)/Humerus...,Humerus_AP_1,0,4095,int16,3240,2774,1,False,/home/xiaofey/xray/xray-master/data/exported_j...
...,...,...,...,...,...,...,...,...,...,...,...
1379,AN00135,/zfs/wficai/xray/xray//Images/AN (135)/Shoulde...,Shoulder_Y_view_3,0,1023,int16,2518,3001,1,False,/home/xiaofey/xray/xray-master/data/exported_j...
1380,AN00176,/zfs/wficai/xray/xray//Images/AN (176)/AP_5.nii,AP_5,371,4095,int16,1769,2079,1,False,/home/xiaofey/xray/xray-master/data/exported_j...
1381,AN00176,/zfs/wficai/xray/xray//Images/AN (176)/AXIAL_3...,AXIAL_3,749,4095,int16,1863,1546,1,False,/home/xiaofey/xray/xray-master/data/exported_j...
1382,AN00176,/zfs/wficai/xray/xray//Images/AN (176)/AP_INT_...,AP_INT_ROT_1,482,4095,int16,1870,1755,1,False,/home/xiaofey/xray/xray-master/data/exported_j...


# Create data json for labelbox

In [7]:
def create_record(row):
    filename = row['JPG_PATH'].split('/')[-1]

    record = dict(
        externalId = filename[:-4],
        imageUrl = f"https://fast-videos.s3.us-east-2.amazonaws.com/phf/cobra_prelim_set/{filename}"
    )
    
    return record
    
create_record(next(df.iterrows())[1])

{'externalId': 'AN00121_Shoulder_L_1',
 'imageUrl': 'https://fast-videos.s3.us-east-2.amazonaws.com/phf/cobra_prelim_set/AN00121_Shoulder_L_1.jpg'}

In [8]:
print(df.columns)

Index(['PATIENT_ID', 'FULL_PATH', 'FILENAME_VIEW', 'MIN', 'MAX', 'DATA_TYPE',
       'WIDTH', 'HEIGHT', 'NUM_CHANNELS', 'ROI', 'JPG_PATH'],
      dtype='object')


In [8]:
df['ROI']

0       False
1        True
2       False
3       False
4       False
        ...  
1379    False
1380    False
1381    False
1382    False
1383    False
Name: ROI, Length: 1384, dtype: bool

In [9]:
records = []
for ix, row in df.iterrows():
    # if not row['ROI'] and row['is_valid']:
    if not row['ROI']: 
        records.append(create_record(row))

In [10]:
records[:5]

[{'externalId': 'AN00121_Shoulder_L_1',
  'imageUrl': 'https://fast-videos.s3.us-east-2.amazonaws.com/phf/cobra_prelim_set/AN00121_Shoulder_L_1.jpg'},
 {'externalId': 'AN00121_Shoulder_L_2',
  'imageUrl': 'https://fast-videos.s3.us-east-2.amazonaws.com/phf/cobra_prelim_set/AN00121_Shoulder_L_2.jpg'},
 {'externalId': 'AN00121_Shoulder_L_3',
  'imageUrl': 'https://fast-videos.s3.us-east-2.amazonaws.com/phf/cobra_prelim_set/AN00121_Shoulder_L_3.jpg'},
 {'externalId': 'AN00308_Humerus_AP_1',
  'imageUrl': 'https://fast-videos.s3.us-east-2.amazonaws.com/phf/cobra_prelim_set/AN00308_Humerus_AP_1.jpg'},
 {'externalId': 'AN00308_Humerus_LAT_3',
  'imageUrl': 'https://fast-videos.s3.us-east-2.amazonaws.com/phf/cobra_prelim_set/AN00308_Humerus_LAT_3.jpg'}]

In [11]:
len(records)

1146

In [12]:
with open(lbx_path, 'w') as f:
    json.dump(records, f)